In [21]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [3]:
import torch, open3d, pandas, tqdm, matplotlib, sklearn
print("All imports OK, CUDA:", torch.version.cuda)

All imports OK, CUDA: 11.8


In [1]:
!pip install torch==2.3.1+cu118 torchvision==0.18.1+cu118 torchaudio==2.3.1+cu118 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 839.6/839.6 MB 561.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 131.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 121.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 137.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 115.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 728.5/728.5 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/

In [2]:
!pip install open3d pandas tqdm matplotlib scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.7/447.7 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 75.2 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


In [5]:
!apt-get install ninja-build
!pip install ninja

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  ninja-build
0 upgraded, 1 newly installed, 0 to remove and 41 not upgraded.
Need to get 111 kB of archives.
After this operation, 358 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 ninja-build amd64 1.10.1-1 [111 kB]
Fetched 111 kB in 1s (95.5 kB/s)
Selecting previously unselected package ninja-build.
(Reading database ... 125082 files and directories currently installed.)
Preparing to unpack .../ninja-build_1.10.1-1_amd64.deb ...
Unpacking ninja-build (1.10.1-1) ...
Setting up ninja-build (1.10.1-1) ...
Processing triggers for man-db (2.10.2-1) ...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 16.0 MB/s eta 0:00:00


In [8]:
# 0) Sanity: you should see a GPU
!nvidia-smi

Sun Nov  9 19:38:48 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [9]:
# 1) Install CUDA 11.8 toolkit into the VM
!wget -q https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/cuda-keyring_1.1-1_all.deb
!dpkg -i cuda-keyring_1.1-1_all.deb
!apt-get update -qq
!apt-get install -y cuda-toolkit-11-8


(Reading database ... 125094 files and directories currently installed.)
Preparing to unpack cuda-keyring_1.1-1_all.deb ...
Unpacking cuda-keyring (1.1-1) over (1.1-1) ...
Setting up cuda-keyring (1.1-1) ...
W: Failed to fetch https://r2u.stat.illinois.edu/ubuntu/dists/jammy/InRelease  Could not connect to r2u.stat.illinois.edu:443 (192.17.190.167), connection timed out
W: Some index files failed to download. They have been ignored, or old ones used instead.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  ca-certificates-java cuda-cccl-11-8 cuda-command-line-tools-11-8
  cuda-compiler-11-8 cuda-cudart-11-8 cuda-cudart-dev-11-8 cuda-cuobjdump-11-8
  cuda-cupti-11-8 cuda-cupti-dev-11-8 cuda-cuxxfilt-11-8
  cuda-documentation-11-8 cuda-driver-dev-11-8 cuda-gdb-11-8
  cuda-libraries-11-8 cuda-libraries-dev-11-8 cuda-memcheck-11-8
  cuda-nsight-11-8 cuda-nsight-compute-11-8 cuda-nsight-sy

In [11]:
!export CUDA_HOME=/usr/local/cuda-11.8 && \
export PATH=$CUDA_HOME/bin:$PATH && \
export LD_LIBRARY_PATH=$CUDA_HOME/lib64:$LD_LIBRARY_PATH && \
nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [12]:
# 3) Install matching PyTorch (cu118) and build deps
!pip install -q torch==2.3.1+cu118 torchvision==0.18.1+cu118 torchaudio==2.3.1+cu118 -f https://download.pytorch.org/whl/torch_stable.html
!apt-get install -y ninja-build
!pip install -q ninja


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ninja-build is already the newest version (1.10.1-1).
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.


In [13]:
# 4) Clean any failed builds
!rm -rf /root/.cache/torch_extensions


In [14]:
# 5) Build ONLY the PointNet2 CUDA ops (subdirectory install)
#    IMPORTANT: run in the *same cell* where CUDA_HOME is exported (or re-export it).
%%bash
export CUDA_HOME=/usr/local/cuda-11.8
export PATH=$CUDA_HOME/bin:$PATH
export LD_LIBRARY_PATH=$CUDA_HOME/lib64:$LD_LIBRARY_PATH
pip install -v "git+https://github.com/erikwijmans/Pointnet2_PyTorch.git#egg=pointnet2_ops&subdirectory=pointnet2_ops_lib"


Using pip 24.1.2 from /usr/local/lib/python3.12/dist-packages/pip (python 3.12)
  Cloning https://github.com/erikwijmans/Pointnet2_PyTorch.git to /tmp/pip-install-br1wgcue/pointnet2-ops_39be4638e18640648079019e66adeb1f
  Resolved https://github.com/erikwijmans/Pointnet2_PyTorch.git to commit b5ceb6d9ca0467ea34beb81023f96ee82228f626
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pointnet2_ops: filename=pointnet2_ops-3.0.0-cp312-cp312-linux_x86_64.whl size=8821280 sha256=a0afdabf22747b2a08d62c6d6277bfac22a6c4f4cf40630ba92e70132620aa8f
  Stored in directory: /tmp/pip-ephem-wheel-cache-iv2gjwyl/wheels/e3/8a/f5/93316dcd272a2b99b20f871c7465287c7325c92039fb555162
Successfully built pointnet2_ops


  Running command git version
  git version 2.34.1
  Running command git clone --filter=blob:none https://github.com/erikwijmans/Pointnet2_PyTorch.git /tmp/pip-install-br1wgcue/pointnet2-ops_39be4638e18640648079019e66adeb1f
  Cloning into '/tmp/pip-install-br1wgcue/pointnet2-ops_39be4638e18640648079019e66adeb1f'...
  Running command git rev-parse HEAD
  b5ceb6d9ca0467ea34beb81023f96ee82228f626
  Running command git rev-parse HEAD
  b5ceb6d9ca0467ea34beb81023f96ee82228f626
  Running command python setup.py egg_info
  running egg_info
  creating /tmp/pip-pip-egg-info-27by_h1b/pointnet2_ops.egg-info
  writing /tmp/pip-pip-egg-info-27by_h1b/pointnet2_ops.egg-info/PKG-INFO
  writing dependency_links to /tmp/pip-pip-egg-info-27by_h1b/pointnet2_ops.egg-info/dependency_links.txt
  writing requirements to /tmp/pip-pip-egg-info-27by_h1b/pointnet2_ops.egg-info/requires.txt
  writing top-level names to /tmp/pip-pip-egg-info-27by_h1b/pointnet2_ops.egg-info/top_level.txt
  writing manifest file '/tm

In [15]:
# 6) Verify versions and import
import os, torch
print("torch:", torch.__version__, "torch.cuda:", torch.version.cuda, "cuda.is_available:", torch.cuda.is_available())
print("CUDA_HOME:", os.environ.get("CUDA_HOME"))

from pointnet2_ops.pointnet2_modules import PointnetFPModule, PointnetSAModule
print("Import OK")


torch: 2.3.1+cu118 torch.cuda: 11.8 cuda.is_available: True
CUDA_HOME: None
Import OK


In [7]:
!pip install -v "git+https://github.com/erikwijmans/Pointnet2_PyTorch.git#egg=pointnet2_ops&subdirectory=pointnet2_ops_lib"

Using pip 24.1.2 from /usr/local/lib/python3.12/dist-packages/pip (python 3.12)
  Cloning https://github.com/erikwijmans/Pointnet2_PyTorch.git to /tmp/pip-install-e6gs3q8q/pointnet2-ops_d404234a2b8248d8b95f2db113c03727
  Running command git version
  git version 2.34.1
  Running command git clone --filter=blob:none https://github.com/erikwijmans/Pointnet2_PyTorch.git /tmp/pip-install-e6gs3q8q/pointnet2-ops_d404234a2b8248d8b95f2db113c03727
  Cloning into '/tmp/pip-install-e6gs3q8q/pointnet2-ops_d404234a2b8248d8b95f2db113c03727'...
  Running command git rev-parse HEAD
  b5ceb6d9ca0467ea34beb81023f96ee82228f626
  Resolved https://github.com/erikwijmans/Pointnet2_PyTorch.git to commit b5ceb6d9ca0467ea34beb81023f96ee82228f626
  Running command git rev-parse HEAD
  b5ceb6d9ca0467ea34beb81023f96ee82228f626
  Running command python setup.py egg_info
  running egg_info
  creating /tmp/pip-pip-egg-info-wue240fq/pointnet2_ops.egg-info
  writing /tmp/pip-pip-egg-info-wue240fq/pointnet2_ops.egg-inf

In [3]:
%cd /content/drive/MyDrive/diplomova_praca/pointnet2/
!git clone https://github.com/erikwijmans/Pointnet2_PyTorch.git

/content/drive/MyDrive/diplomova_praca/pointnet2
Cloning into 'Pointnet2_PyTorch'...
remote: Enumerating objects: 710, done.
remote: Counting objects: 100% (151/151), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 710 (delta 114), reused 99 (delta 99), pack-reused 559 (from 1)
Receiving objects: 100% (710/710), 978.92 KiB | 11.00 MiB/s, done.
Resolving deltas: 100% (402/402), done.


In [20]:
%cd /content/drive/MyDrive/diplomova_praca/pointnet2/Pointnet2_PyTorch/
!pip install -e .

/content/drive/MyDrive/diplomova_praca/pointnet2/Pointnet2_PyTorch
Obtaining file:///content/drive/MyDrive/diplomova_praca/pointnet2/Pointnet2_PyTorch
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for pointnet2 (pyproject.toml) ... done
  Created wheel for pointnet2: filename=pointnet2-3.0.0-0.editable-py3-none-any.whl size=2755 sha256=2431c622383498cd702fec4436d0de35ba975c01a71020e067049c2b9806d695
  Stored in directory: /tmp/pip-ephem-wheel-cache-aosozojt/wheels/4c/69/98/64a790cd7394b6266fd5f30abf17c187d0508e779f7be4bd25
Successfully built pointnet2
  Attempting uninstall: pointnet2
    Found existing installation: pointnet2 3.0.0
    Uninstalling pointnet2-3.0.0:
      Successfully uninstalled pointnet2-3.0.0


In [25]:
%cd /content/drive/MyDrive/diplomova_praca/pointnet2/Pointnet2_PyTorch/pointnet2_ops_lib
!pip install -e .

/content/drive/MyDrive/diplomova_praca/pointnet2/Pointnet2_PyTorch/pointnet2_ops_lib
Obtaining file:///content/drive/MyDrive/diplomova_praca/pointnet2/Pointnet2_PyTorch/pointnet2_ops_lib
  Preparing metadata (setup.py) ... done
  Running setup.py develop for pointnet2_ops
    error: subprocess-exited-with-error
    
    × python setup.py develop did not run successfully.
    │ exit code: 1
    ╰─> See above for output.
    
    note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× python setup.py develop did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [16]:
%cd /content/drive/MyDrive/diplomova_praca/pointnet2/Pointnet2_PyTorch/pointnet2_ops_lib/pointnet2_ops/
!ls

/content/drive/MyDrive/diplomova_praca/pointnet2/Pointnet2_PyTorch/pointnet2_ops_lib/pointnet2_ops
_ext-src     pointnet2_modules.py  __pycache__
__init__.py  pointnet2_utils.py    _version.py


In [17]:
import sys
sys.path.append('/content/drive/MyDrive/diplomova_praca/pointnet2/Pointnet2_PyTorch/pointnet2_ops_lib')

from pointnet2_ops.pointnet2_modules import PointnetFPModule, PointnetSAModule

print("Import successful.")


Import successful.


In [16]:
!pip wheel "git+https://github.com/erikwijmans/Pointnet2_PyTorch.git#egg=pointnet2_ops&subdirectory=pointnet2_ops_lib" -w /content/drive/MyDrive/diplomova_praca/wheel


  Cloning https://github.com/erikwijmans/Pointnet2_PyTorch.git to /tmp/pip-wheel-2uu35mwm/pointnet2-ops_701c2d523cbc4c4caa02024ab704f445
  Running command git clone --filter=blob:none --quiet https://github.com/erikwijmans/Pointnet2_PyTorch.git /tmp/pip-wheel-2uu35mwm/pointnet2-ops_701c2d523cbc4c4caa02024ab704f445
  Resolved https://github.com/erikwijmans/Pointnet2_PyTorch.git to commit b5ceb6d9ca0467ea34beb81023f96ee82228f626
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 782.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 860.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193

In [ ]:
import os
import ast
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import open3d as o3d
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from pointnet2_ops.pointnet2_modules import PointnetFPModule, PointnetSAModuleMSG


# =====================
#   HYPERPARAMETRE
# =====================
BATCH_SIZE = 2
LR = 0.0025
WEIGHT_DECAY = 1e-4
LR_DECAY_FACTOR = 0.5
LR_DECAY_STEP = 20
EPOCHS = 100
N_CLASSES = 2
N_POINTS = 175620
TRAIN_RATIO, VAL_RATIO, TEST_RATIO = 0.7, 0.1, 0.2
SEED = 42

# =====================
#   NASTAVENIA
# =====================
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

DATA_DIR = "/content/gdrive/MyDrive/diplomova_praca/original"
PC_INFO = os.path.join(DATA_DIR, "point_counts.csv")
LABEL_INFO = os.path.join(DATA_DIR, "labels_teaser_before.csv")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# =====================
#   DATASET
# =====================

class BranchSegmentationDataset(Dataset):
    def __init__(self, data_dir, pc_info_path, label_path, n_points):
        self.data_dir = data_dir
        self.pc_info = pd.read_csv(pc_info_path)
        self.labels_df = pd.read_csv(label_path)
        self.n_points = n_points

        # Map tree name (without _merged_teaser.ply) to label array
        self.labels_df['labels'] = self.labels_df['labels'].apply(ast.literal_eval)
        label_dict = dict(zip(self.labels_df['tree'], self.labels_df['labels']))

        self.pc_info = self.pc_info[self.pc_info['file_name'].str.replace('_merged_teaser.ply', '').isin(label_dict.keys())]
        self.files = self.pc_info['file_name'].tolist()
        self.labels = [label_dict[f.replace('_merged_teaser.ply', '')] for f in self.files]

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file_name = self.files[idx]
        label_array = np.array(self.labels[idx], dtype=np.int64)

        pcd = o3d.io.read_point_cloud(os.path.join(self.data_dir, file_name))
        pts = np.asarray(pcd.points, dtype=np.float32)

        # Match labels and points (downsample/upsample if needed)
        min_len = min(len(pts), len(label_array))
        pts, label_array = pts[:min_len], label_array[:min_len]

        if min_len > self.n_points:
            choice = np.random.choice(min_len, self.n_points, replace=False)
        else:
            choice = np.random.choice(min_len, self.n_points, replace=True)

        pts = pts[choice, :]
        label_array = label_array[choice]

        pts = torch.tensor(pts).float().T  # (3, N)
        labels = torch.tensor(label_array).long()  # (N,)
        return pts, labels


# =====================
#   MODEL (PointNet++)
# =====================
class PointNet2SemSegMSG(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.sa1 = PointnetSAModuleMSG(
            npoint=1024,
            radii=[0.1, 0.2, 0.4],
            nsamples=[16, 32, 128],
            mlps=[[0, 32, 32, 64], [0, 64, 64, 128], [0, 64, 96, 128]],
            use_xyz=True
        )
        self.sa2 = PointnetSAModuleMSG(
            npoint=256,
            radii=[0.4, 0.8],
            nsamples=[32, 64],
            mlps=[[320, 128, 128, 256], [320, 128, 196, 256]],
            use_xyz=True
        )
        self.fp1 = PointnetFPModule(mlp=[512 + 320, 256, 256])  # correct: 832 input channels
        self.fp2 = PointnetFPModule(mlp=[256 + 3, 256, 128, num_classes])

    def forward(self, xyz):
        xyz = xyz.transpose(1, 2).contiguous()  # (B, N, 3)
        l1_xyz, l1_features = self.sa1(xyz, None)
        l2_xyz, l2_features = self.sa2(l1_xyz, l1_features)

        l1_features = self.fp1(l1_xyz, l2_xyz, l1_features, l2_features)
        l0_features = self.fp2(xyz, l1_xyz, None, l1_features)

        return l0_features.transpose(1, 2)  # (B, num_classes, N)
# =====================
#   DÁTA A LOADERY
# =====================
dataset = BranchSegmentationDataset(DATA_DIR, PC_INFO, LABEL_INFO, N_POINTS)

train_len = int(TRAIN_RATIO * len(dataset))
val_len = int(VAL_RATIO * len(dataset))
test_len = len(dataset) - train_len - val_len
train_ds, val_ds, test_ds = random_split(dataset, [train_len, val_len, test_len])

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

# =====================
#   TRÉNING
# =====================
model = PointNet2SemSegMSG(num_classes=N_CLASSES).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=LR_DECAY_STEP, gamma=LR_DECAY_FACTOR)

train_losses, val_losses = [], []
train_accs, val_accs = [], []
train_precisions, val_precisions = [], []
train_recalls, val_recalls = [], []
train_f1s, val_f1s = [], []
best_val_loss = float('inf')

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0
    all_preds, all_labels = [], []

    for pts, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        pts, labels = pts.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(pts)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        _, preds = torch.max(outputs, 1)  # (B, N)
        all_preds.extend(preds.cpu().numpy().flatten())
        all_labels.extend(labels.cpu().numpy().flatten())


    # metriky pre tréning
    avg_train_loss = running_loss / len(train_loader)
    train_losses.append(avg_train_loss)
    train_accs.append(accuracy_score(all_labels, all_preds))
    train_precisions.append(precision_score(all_labels, all_preds, average='binary', zero_division=0))
    train_recalls.append(recall_score(all_labels, all_preds, average='binary', zero_division=0))
    train_f1s.append(f1_score(all_labels, all_preds, average='binary', zero_division=0))

    # =====================
    # Validácia
    # =====================
    model.eval()
    val_loss = 0
    val_preds, val_labels_all = [], []

    with torch.no_grad():
        for pts, labels in val_loader:
            pts, labels = pts.to(device), labels.to(device)
            outputs = model(pts)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, preds = torch.max(outputs, 1)
            val_preds.extend(preds.cpu().numpy().flatten())
            val_labels_all.extend(labels.cpu().numpy().flatten())

    avg_val_loss = val_loss / len(val_loader)
    val_losses.append(avg_val_loss)
    val_accs.append(accuracy_score(val_labels_all, val_preds))
    val_precisions.append(precision_score(val_labels_all, val_preds, average='binary', zero_division=0))
    val_recalls.append(recall_score(val_labels_all, val_preds, average='binary', zero_division=0))
    val_f1s.append(f1_score(val_labels_all, val_preds, average='binary', zero_division=0))

    scheduler.step()

    print(f"Epoch {epoch+1}: "
          f"Train Loss={avg_train_loss:.4f}, Val Loss={avg_val_loss:.4f}, "
          f"Train Acc={train_accs[-1]:.3f}, Val Acc={val_accs[-1]:.3f}")
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        metrics = {
          "train_losses": train_losses,
          "val_losses": val_losses,
          "train_accs": train_accs,
          "val_accs": val_accs,
          "train_precisions": train_precisions,
          "val_precisions": val_precisions,
          "train_recalls": train_recalls,
          "val_recalls": val_recalls,
          "train_f1s": train_f1s,
          "val_f1s": val_f1s
        }
        torch.save(metrics, "/content/gdrive/MyDrive/diplomova_praca/training_metrics.pt")

        torch.save(model.state_dict(),
                   "/content/gdrive/MyDrive/diplomova_praca/best_pointnet2_model.pth")
        print(f"Best model saved at epoch {epoch+1} with val loss {avg_val_loss:.4f}")

# =====================
#   TESTOVANIE
# =====================
model.eval()
test_preds, test_labels_all = [], []
with torch.no_grad():
    for pts, labels in tqdm(test_loader, desc="Testing"):
        pts, labels = pts.to(device), labels.to(device)
        outputs = model(pts)
        _, preds = torch.max(outputs, 1)
        test_preds.extend(preds.cpu().numpy().flatten())
        test_labels_all.extend(labels.cpu().numpy().flatten())

test_acc = accuracy_score(test_labels_all, test_preds)
test_prec = precision_score(test_labels_all, test_preds, average='binary', zero_division=0)
test_rec = recall_score(test_labels_all, test_preds, average='binary', zero_division=0)
test_f1 = f1_score(test_labels_all, test_preds, average='binary', zero_division=0)

print(f"\n--- Výsledky na testovacej množine ---")
print(f"Accuracy: {test_acc:.4f}")
print(f"Precision: {test_prec:.4f}")
print(f"Recall:    {test_rec:.4f}")
print(f"F1 score:  {test_f1:.4f}")

# =====================
#   VIZUALIZÁCIA
# =====================
fig, axs = plt.subplots(2, 2, figsize=(12, 8))

# Loss
axs[0, 0].plot(train_losses, label='Train Loss')
axs[0, 0].plot(val_losses, label='Val Loss')
axs[0, 0].set_title('Loss')
axs[0, 0].legend()
axs[0, 0].grid(True)

# Accuracy
axs[0, 1].plot(train_accs, label='Train Acc')
axs[0, 1].plot(val_accs, label='Val Acc')
axs[0, 1].set_title('Accuracy')
axs[0, 1].legend()
axs[0, 1].grid(True)

# Precision
axs[1, 0].plot(train_precisions, label='Train Precision')
axs[1, 0].plot(val_precisions, label='Val Precision')
axs[1, 0].set_title('Precision')
axs[1, 0].legend()
axs[1, 0].grid(True)

# Recall/F1
axs[1, 1].plot(train_recalls, label='Train Recall')
axs[1, 1].plot(val_recalls, label='Val Recall')
axs[1, 1].plot(train_f1s, '--', label='Train F1')
axs[1, 1].plot(val_f1s, '--', label='Val F1')
axs[1, 1].set_title('Recall / F1')
axs[1, 1].legend()
axs[1, 1].grid(True)

plt.tight_layout()
plt.savefig("/content/gdrive/MyDrive/diplomova_praca/metrics_plot_original.png", dpi=300)
plt.show()

In [ ]:
import os
import random
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import open3d as o3d
from pointnet2_ops.pointnet2_modules import PointnetFPModule, PointnetSAModule

# =====================
#   HYPERPARAMETRE
# =====================
BATCH_SIZE = 2
LR = 0.0025
OPTIMIZER = 'Adam'
WEIGHT_DECAY = 1e-4
LR_DECAY_FACTOR = 0.5
LR_DECAY_STEP = 20
EPOCHS = 100
N_CLASSES = 2
N_POINTS = 62673
TRAIN_RATIO, VAL_RATIO, TEST_RATIO = 0.7, 0.1, 0.2
SEED = 42

# =====================
#   NASTAVENIA
# =====================
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

DATA_DIR = "/content/drive/MyDrive/diplomova_praca/normalized_centered_voxelized_models_0.0025"
PC_INFO = os.path.join(DATA_DIR, "point_clouds.xlsx")
LABEL_INFO = os.path.join(DATA_DIR, "voxel_labels_0.0025.xlsx")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# =====================
#   DATASET
# =====================


class BranchDataset(Dataset):
    def __init__(self, data_dir, pc_info_path, label_path, n_points):
        self.data_dir = data_dir
        self.pc_info = pd.read_excel(pc_info_path)
        self.labels_df = pd.read_excel(label_path)
        self.n_points = n_points

        # mapovanie mena súboru na label
        label_dict = dict(zip(self.labels_df['tree'], self.labels_df['labels']))
        self.pc_info = self.pc_info[self.pc_info['file_name'].isin(label_dict.keys())]
        self.pc_info = self.pc_info[self.pc_info['point_count'] >= n_points]

        self.files = self.pc_info['file_name'].tolist()
        self.labels = [label_dict[f] for f in self.files]

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file_name = self.files[idx]
        label = int(self.labels[idx])

        pcd = o3d.io.read_point_cloud(os.path.join(self.data_dir, file_name))
        pts = np.asarray(pcd.points, dtype=np.float32)

        # náhodný sampling (ak viac bodov)
        if pts.shape[0] > self.n_points:
            choice = np.random.choice(pts.shape[0], self.n_points, replace=False)
            pts = pts[choice, :]
        elif pts.shape[0] < self.n_points:
            # doplnenie náhodnými duplikátmi
            choice = np.random.choice(pts.shape[0], self.n_points - pts.shape[0], replace=True)
            pts = np.concatenate([pts, pts[choice, :]], axis=0)

        pts = torch.tensor(pts).float().T  # (3, N)
        return pts, label

# =====================
#   MODEL (PointNet++)
# =====================
class PointNet2ClsMSG(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.sa1 = PointnetSAModule(npoint=1024, radii=[0.1, 0.2], nsamples=[32, 64],
                                    mlps=[[3, 32, 32, 64], [3, 64, 64, 128]])
        self.sa2 = PointnetSAModule(npoint=256, radii=[0.2, 0.4], nsamples=[32, 64],
                                    mlps=[[192, 128, 128, 256], [192, 128, 196, 256]])
        self.sa3 = PointnetSAModule(npoint=None, radii=None, nsamples=None,
                                    mlps=[[512, 256, 512, 1024]], use_xyz=False)

        self.fc = nn.Sequential(
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(256, num_classes)
        )

    def forward(self, xyz):
        xyz = xyz.transpose(1, 2)  # (B, N, 3)
        l1_xyz, l1_features = self.sa1(xyz, None)
        l2_xyz, l2_features = self.sa2(l1_xyz, l1_features)
        l3_xyz, l3_features = self.sa3(l2_xyz, l2_features)
        x = torch.max(l3_features, 2)[0]
        x = self.fc(x)
        return x

# =====================
#   DÁTA A LOADERY
# =====================
dataset = BranchDataset(DATA_DIR, PC_INFO, LABEL_INFO, N_POINTS)

train_len = int(TRAIN_RATIO * len(dataset))
val_len = int(VAL_RATIO * len(dataset))
test_len = len(dataset) - train_len - val_len

train_ds, val_ds, test_ds = random_split(dataset, [train_len, val_len, test_len])

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False)

# =====================
#   TRÉNING
# =====================
model = PointNet2ClsMSG(num_classes=N_CLASSES).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=LR_DECAY_STEP, gamma=LR_DECAY_FACTOR)

train_losses = []
val_losses = []

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0
    for pts, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        pts, labels = pts.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(pts)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    avg_train_loss = running_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    # validácia
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for pts, labels in val_loader:
            pts, labels = pts.to(device), labels.to(device)
            outputs = model(pts)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

    avg_val_loss = val_loss / len(val_loader)
    val_losses.append(avg_val_loss)
    scheduler.step()

    print(f"Epoch {epoch+1}: Train Loss = {avg_train_loss:.4f}, Val Loss = {avg_val_loss:.4f}")

# =====================
#   TESTOVANIE PRESNOSTI
# =====================
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for pts, labels in tqdm(test_loader, desc="Testing"):
        pts, labels = pts.to(device), labels.to(device)
        outputs = model(pts)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Presnosť na testovacej množine: {accuracy:.2f}%")

# =====================
#   ULOŽENIE MODELU
# =====================
torch.save(model.state_dict(), "/content/drive/MyDrive/diplomova_praca/pointnet2_pruned_classifier_voxel_0_0025.pth")
print("Model uložený do pointnet2_pruned_classifier.pth")

# =====================
#   VIZUALIZÁCIA STRÁT
# =====================
plt.figure(figsize=(8, 5))
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Priebeh stratovej funkcie')
plt.legend()
plt.grid(True)
plt.tight_layout()
# uloženie grafu
plt.savefig("/content/drive/MyDrive/diplomova_praca/loss_plot_voxel_0_0025.png", dpi=300)
plt.show()

ModuleNotFoundError: No module named 'pointnet2_ops'

In [ ]:
!pip show pointnet2
!ls /usr/local/lib/python*/dist-packages/pointnet2_ops*

Name: pointnet2
Version: 3.0.0
Summary: 
Home-page: 
Author: Erik Wijmans
Author-email: 
License: 
Location: /usr/local/lib/python3.12/dist-packages
Requires: hydra-core, pytorch-lightning
Required-by: 
ls: cannot access '/usr/local/lib/python*/dist-packages/pointnet2_ops*': No such file or directory
